In [69]:
from random import shuffle
import glob,h5py, cv2
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
duration = 24.2
fps = 25
subject = 'Blake2'
num_frames = int(duration*fps) # axis = 1 dimension, also the # of frames are going to be padded to this value.
filename = 'Datasets/'+subject+'_RAmap.hdf5'
print('File to be created: ' + filename)

File to be created: Datasets/Blake2_RAmap.hdf5


In [70]:
# load labels
trainy = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/*.txt'
trainy_files = sorted(glob.glob(trainy))
print('Num. of label files: '+str(len(trainy_files)))
for i in range(0,5):
    print(str(i+1)+'. ' +trainy_files[i])

Num. of label files: 23
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11010003_1617741768_1.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11010003_1617741768_2.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11010003_1617741768_3.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11010003_1617741768_4.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11010003_1617741768_5.txt


In [71]:
# load videos
trainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/rangeDOA/*.avi'
train_files2 = sorted(glob.glob(trainpath))
len(train_files2)
train_files2[0]
fname_ls = []
train_files = []
for i in range(0,len(trainy_files)):
#     fname = trainy_files[i][-23:].replace("txt","avi") # -23 for nonbinary labels
#     fname = trainy_files[i][-30:].replace("txt","avi") # -23 for nonbinary labels
#     fname = fname.replace('_binary','')
#     fname = trainy_files[i][-9:].replace("txt","avi") # blake
    slash_idx = trainy_files[i].rfind('/')
    fname = trainy_files[i][slash_idx:].replace("txt","avi")
    fname_ls.append(fname)
    idx = [j for j, s in enumerate(train_files2) if fname in s] #train_files2.index(fname) 
    if len(idx) == 1:
        train_files.append(train_files2[idx[0]])
    else:
        print('2 identical filenames at i=',str(i))
print('Num. of label files: '+str(len(train_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+train_files[i])

Num. of label files: 23
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11010003_1617741768_1.avi
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11010003_1617741768_2.avi
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11010003_1617741768_3.avi
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11010003_1617741768_4.avi
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11010003_1617741768_5.avi


In [72]:
def FrameCapture_backup(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
  
    # checks whether frames were extracted 
    success = 1
    frames = []
    while vidObj.isOpened: 
  
        # vidObj object calls read 
        # function extract frames 
        success, frame = vidObj.read() 
        resize = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        frames.append(resize) # .resize(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [73]:
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [74]:
def pad_video(video):
    if video.shape[0] < num_frames:
        while video.shape[0] < num_frames:
            video_padded = np.concatenate((video, video[-(num_frames-len(video)):]), axis = 0)
            if video_padded.shape[0] == num_frames:
                break
    else:
        video_padded = video[:num_frames]
#     video_padded = video[:-1]
    return video_padded

In [75]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [76]:
videos = []
video_di = []
cnt = 1
for video in train_files:
    
    clear_output(wait=True)
    print('Loading video '+str(cnt)+'/'+str(len(train_files))+ ' '+video)#,end="\r")
    video_di = pad_video(FrameCapture(video))
#     video_di = FrameCapture(video)
    videos.append(video_di)
        
    
    cnt += 1
videos = np.array(videos)   
# x_train = np.array(np.concatenate(x_train))

videos = videos/255. # normalize
print(videos.shape)

Loading video 23/23 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11050003_1617742722_4.avi
(23, 605, 128, 128, 3)


In [77]:
def pad_labels(yx):
    numpad = duration*fps - yx.shape[1]
    pads = np.repeat(yx[0,-1],numpad)
    padded = np.expand_dims(np.append(yx,pads),0)
    return padded

In [78]:
labels = []

cnt = 1
for file in trainy_files:
    
    clear_output(wait=True)
    print('Loading training labels #'+str(cnt))
    yx = load_file(file)
    if yx.shape[1] < duration*fps:
        yx = pad_labels(yx)
    idx = np.linspace(0,yx.shape[1]-1,num_frames).astype(int)
    labels.append(yx[0,idx])
#     labels.append(yx)
    
    cnt += 1
labels = np.array(labels)    
# labels = to_categorical(np.swapaxes(labels,1,2))
labels = to_categorical(labels)
idx = np.linspace(0,labels.shape[1]-1,num_frames).astype(int)
labels = labels[:,idx,:]
print(labels.shape)

Loading training labels #23
(23, 605, 19)


#### Train-Test Split

In [79]:
x_train, x_test, y_train, y_test = train_test_split(videos, labels, test_size=0.2, random_state=1)

# if Reshape == 1:
#     x_train = x_train.reshape(-1,fps,x_train.shape[2],x_train.shape[3],x_train.shape[4])
#     x_test = x_test.reshape(-1,fps,x_test.shape[2],x_test.shape[3],x_test.shape[4])
#     y_train = y_train.reshape(-1,fps,y_train.shape[2])
#     y_test = y_test.reshape(-1,fps,y_test.shape[2])
    
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print('# of frames in a class:')
Counter(np.concatenate(np.argmax(y_train, axis=2),axis=0))

(18, 605, 128, 128, 3)
(18, 605, 19)
(5, 605, 128, 128, 3)
(5, 605, 19)
# of frames in a class:


Counter({0: 2994,
         1: 2449,
         2: 1107,
         13: 236,
         14: 276,
         15: 266,
         3: 957,
         10: 176,
         11: 198,
         12: 187,
         4: 237,
         5: 257,
         6: 271,
         7: 304,
         8: 300,
         9: 313,
         16: 122,
         17: 123,
         18: 117})

In [80]:
f = h5py.File(filename, mode='w')
f.create_dataset("train_img", x_train.shape, np.uint8)
f.create_dataset("test_img", x_test.shape, np.uint8)  
f.create_dataset("train_labels", y_train.shape, np.uint8)
f.create_dataset("test_labels", y_test.shape, np.uint8)

f["train_img"][...] = x_train
f["test_img"][...] = x_test
f["train_labels"][...] = y_train
f["test_labels"][...] = y_test
f.close()
print(filename+' created.')

Datasets/Blake2_RAmap.hdf5 created.
